In [12]:
import numpy as np
import pandas as pd
#import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
# from statsmodels.stats.proportion import proportions_ztest
# from statsmodels.stats.proportion import proportions_chisquare
from IPython.display import display, HTML
from scipy.stats import kruskal

import sys
import logging
logging.basicConfig(level=logging.CRITICAL)

from pytqs import tqs
import bytedtqs
from datetime import date, timedelta

In [13]:
# appId = "eL38NWjpAMXlC0ifCoo71wk27zE6CS9YQismEcwVH5oNKbJf"
# appKey = "VeeOKtXLxjGWE2OyIGkuqWRT33isyxtHH1xlxakeFzId6SMB"
# user_id = 'wuweiwei.1012'

In [14]:
def two_proprotions_test(success_a, size_a, success_b, size_b):
    prop_a = success_a / size_a
    prop_b = success_b / size_b
    prop_pooled = (success_a + success_b) / (size_a + size_b)
    var = prop_pooled * (1 - prop_pooled) * (1 / size_a + 1 / size_b)
    zscore = np.abs(prop_b - prop_a) / np.sqrt(var)
    one_side = 1 - stats.norm(loc = 0, scale = 1).cdf(zscore)
    pvalue = one_side * 2
    return zscore, pvalue

### 获取实验数据

In [32]:
all_data = pd.read_csv('/home/tiger/archived-data/aeolus-data/20210310/11/39616345-课程实验-BKT模型-查询6.csv', encoding = 'gb18030')

In [33]:
all_data['fluctuate_kp_ratio'] = all_data['answers_fluctuate_kp_video_review'] / all_data['answers_kp_video_review']
all_data.head()

,user_id,user_group,abtest_dt,first_milestone_name,new_user,new_user_vip,d0_finish_lessons,d0_study_duration,d1_study_duration,d2_study_duration,d3_study_duration,d4_study_duration,d5_study_duration,d6_study_duration,d7_study_duration,answers_kp_video_review,answers_fluctuate_kp_video_review,fluctuate_kp_ratio
0,2938318424644583,对照组,2021-02-19,B1中,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NaN
1,4328108414994990,对照组,2021-03-07,A0,0,0,2.0,627.0,1924.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NaN
2,4486430632708711,实验组,2021-02-19,A1初,1,0,0.0,0.0,0.0,959.0,2068.0,0.0,0.0,0.0,0.0,0,0,NaN
3,70792815059032,对照组,2021-02-20,A0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NaN
4,1719257237302564,实验组,2021-02-20,未定级,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NaN


In [34]:
len(all_data)

82525

### 新用户转化留存

In [18]:
### 预处理数据
#1. 填充空值数据
#2. 增加新字段

# all_data = all_data[~all_data.milestone_name.isin(['未定级'])]

new_user_data = all_data[all_data.new_user == 1]

new_user_data.loc[(new_user_data['d0_study_duration'] > 0), 'd0_study'] = 1
new_user_data.loc[(new_user_data['d0_study_duration'] == 0), 'd0_study'] = 0

new_user_data.loc[(new_user_data['d0_study_duration'] >= 600), 'd0_study_10min'] = 1
new_user_data.loc[(new_user_data['d0_study_duration'] < 600), 'd0_study_10min'] = 0

new_user_data.loc[(new_user_data['d1_study_duration'] >= 600), 'd1_study_10min'] = 1
new_user_data.loc[(new_user_data['d1_study_duration'] < 600), 'd1_study_10min'] = 0

new_user_data.loc[(new_user_data['d2_study_duration'] >= 600), 'd2_study_10min'] = 1
new_user_data.loc[(new_user_data['d2_study_duration'] < 600), 'd2_study_10min'] = 0

new_user_data.loc[(new_user_data['d3_study_duration'] >= 600), 'd3_study_10min'] = 1
new_user_data.loc[(new_user_data['d3_study_duration'] < 600), 'd3_study_10min'] = 0

new_user_data.loc[(new_user_data['d4_study_duration'] >= 600), 'd4_study_10min'] = 1
new_user_data.loc[(new_user_data['d4_study_duration'] < 600), 'd4_study_10min'] = 0

# review_test_data.head()


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

#### 全量用户 

In [19]:
def ab_summary_data(data, user_group):
    test_summary = pd.DataFrame(columns = ["User_Group", "Metrics", "Control", "Test", "Delta", "p_value","Significance"])
    
    test_data = data.loc[data.user_group == '实验组']
    ctrl_data = data.loc[data.user_group == '对照组']

    dnu_t = len(test_data)
    dnu_c = len(ctrl_data)

    # 新增用户
    test_summary = test_summary.append({"Metrics": '实验用户', "Control": '{:,}'.format(dnu_c), "Test": '{:,}'.format(dnu_t), "Delta": '-', "p_value": '-', "Significance": '-'}, ignore_index = True)

    # d0学习率(>0)
    d0_study_0m_t = sum(test_data.d0_study)
    d0_study_0m_c = sum(ctrl_data.d0_study)
    
    d0_st_0m_rate_t = round(d0_study_0m_t / dnu_t, 4)
    d0_st_0m_rate_c = round(d0_study_0m_c / dnu_c, 4)
    
    d0_st_0m_rate_delta = d0_st_0m_rate_t / d0_st_0m_rate_c - 1 if d0_st_0m_rate_c > 0 else np.nan
    d0_st_0m_rate_p_value = two_proprotions_test(d0_study_0m_t, dnu_t, d0_study_0m_c, dnu_c)[1]
    d0_st_0m_rate_sig =  "Significant" if d0_st_0m_rate_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": 'd0学习率(>0m)', "Control": "{:.2%}".format(d0_st_0m_rate_c), "Test": "{:.2%}".format(d0_st_0m_rate_t), "Delta": "{:.2%}".format(d0_st_0m_rate_delta), "p_value": round(d0_st_0m_rate_p_value, 4), "Significance": d0_st_0m_rate_sig}, ignore_index = True)

    
    # d0学习率(>=10m)
    d0_study_10m_t = sum(test_data.d0_study_10min)
    d0_study_10m_c = sum(ctrl_data.d0_study_10min)
    
    d0_st_10m_rate_t = round(d0_study_10m_t / dnu_t, 4)
    d0_st_10m_rate_c = round(d0_study_10m_c / dnu_c, 4)
    
    d0_st_10m_rate_delta = d0_st_10m_rate_t / d0_st_10m_rate_c - 1 if d0_st_10m_rate_c > 0 else np.nan
    d0_st_10m_rate_p_value = two_proprotions_test(d0_study_10m_t, dnu_t, d0_study_10m_c, dnu_c)[1]
    d0_st_10m_rate_sig =  "Significant" if d0_st_10m_rate_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": 'd0有效学习率(>10m)', "Control": "{:.2%}".format(d0_st_10m_rate_c), "Test": "{:.2%}".format(d0_st_10m_rate_t), "Delta": "{:.2%}".format(d0_st_10m_rate_delta), "p_value": round(d0_st_10m_rate_p_value, 4), "Significance": d0_st_10m_rate_sig}, ignore_index = True)

    # d1学习率(>=10m)
    d1_study_10m_t = sum(test_data.d1_study_10min)
    d1_study_10m_c = sum(ctrl_data.d1_study_10min)
    
    d1_st_10m_rate_t = round(d1_study_10m_t / dnu_t, 4)
    d1_st_10m_rate_c = round(d1_study_10m_c / dnu_c, 4)
    
    d1_st_10m_rate_delta = d1_st_10m_rate_t / d1_st_10m_rate_c - 1 if d1_st_10m_rate_c > 0 else np.nan
    d1_st_10m_rate_p_value = two_proprotions_test(d1_study_10m_t, dnu_t, d1_study_10m_c, dnu_c)[1]
    d1_st_10m_rate_sig =  "Significant" if d1_st_10m_rate_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": 'd1有效学习率(>10m)', "Control": "{:.2%}".format(d1_st_10m_rate_c), "Test": "{:.2%}".format(d1_st_10m_rate_t), "Delta": "{:.2%}".format(d1_st_10m_rate_delta), "p_value": round(d1_st_10m_rate_p_value, 4), "Significance": d1_st_10m_rate_sig}, ignore_index = True)

    # d2学习率(>=10m)
    d2_study_10m_t = sum(test_data.d2_study_10min)
    d2_study_10m_c = sum(ctrl_data.d2_study_10min)
    
    d2_st_10m_rate_t = round(d2_study_10m_t / dnu_t, 4)
    d2_st_10m_rate_c = round(d2_study_10m_c / dnu_c, 4)
    
    d2_st_10m_rate_delta = d2_st_10m_rate_t / d2_st_10m_rate_c - 1 if d2_st_10m_rate_c > 0 else np.nan
    d2_st_10m_rate_p_value = two_proprotions_test(d2_study_10m_t, dnu_t, d2_study_10m_c, dnu_c)[1]
    d2_st_10m_rate_sig =  "Significant" if d2_st_10m_rate_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": 'd2有效学习率(>10m)', "Control": "{:.2%}".format(d2_st_10m_rate_c), "Test": "{:.2%}".format(d2_st_10m_rate_t), "Delta": "{:.2%}".format(d2_st_10m_rate_delta), "p_value": round(d2_st_10m_rate_p_value, 4), "Significance": d2_st_10m_rate_sig}, ignore_index = True)

    # d3学习率(>=10m)
    d3_study_10m_t = sum(test_data.d3_study_10min)
    d3_study_10m_c = sum(ctrl_data.d3_study_10min)
    
    d3_st_10m_rate_t = round(d3_study_10m_t / dnu_t, 4)
    d3_st_10m_rate_c = round(d3_study_10m_c / dnu_c, 4)
    
    d3_st_10m_rate_delta = d3_st_10m_rate_t / d3_st_10m_rate_c - 1 if d3_st_10m_rate_c > 0 else np.nan
    d3_st_10m_rate_p_value = two_proprotions_test(d3_study_10m_t, dnu_t, d3_study_10m_c, dnu_c)[1]
    d3_st_10m_rate_sig =  "Significant" if d3_st_10m_rate_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": 'd3有效学习率(>10m)', "Control": "{:.2%}".format(d3_st_10m_rate_c), "Test": "{:.2%}".format(d3_st_10m_rate_t), "Delta": "{:.2%}".format(d3_st_10m_rate_delta), "p_value": round(d3_st_10m_rate_p_value, 4), "Significance": d3_st_10m_rate_sig}, ignore_index = True)

    # d4学习率(>=10m)
    d4_study_10m_t = sum(test_data.d4_study_10min)
    d4_study_10m_c = sum(ctrl_data.d4_study_10min)
    
    d4_st_10m_rate_t = round(d4_study_10m_t / dnu_t, 4)
    d4_st_10m_rate_c = round(d4_study_10m_c / dnu_c, 4)
    
    d4_st_10m_rate_delta = d4_st_10m_rate_t / d4_st_10m_rate_c - 1 if d4_st_10m_rate_c > 0 else np.nan
    d4_st_10m_rate_p_value = two_proprotions_test(d4_study_10m_t, dnu_t, d4_study_10m_c, dnu_c)[1]
    d4_st_10m_rate_sig =  "Significant" if d4_st_10m_rate_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": 'd4有效学习率(>10m)', "Control": "{:.2%}".format(d4_st_10m_rate_c), "Test": "{:.2%}".format(d4_st_10m_rate_t), "Delta": "{:.2%}".format(d4_st_10m_rate_delta), "p_value": round(d4_st_10m_rate_p_value, 4), "Significance": d4_st_10m_rate_sig}, ignore_index = True)

    # d6转化
    test_data_tmp = data.loc[(data.user_group == '实验组') & (data.abtest_dt <= (date.today() - timedelta(days=7)).strftime('%Y-%m-%d'))]
    ctrl_data_tmp = data.loc[(data.user_group == '对照组') & (data.abtest_dt <= (date.today() - timedelta(days=7)).strftime('%Y-%m-%d'))]
    dnu_t_tmp = len(test_data_tmp)
    dnu_c_tmp = len(ctrl_data_tmp)
    
    vip_t = test_data_tmp.new_user_vip.sum()
    vip_c = ctrl_data_tmp.new_user_vip.sum()
    
    d6_cr_t = round(vip_t / dnu_t_tmp, 4)
    d6_cr_c = round(vip_c / dnu_c_tmp, 4)
    d6_cr_delta = d6_cr_t / d6_cr_c - 1 if d6_cr_c > 0 else np.nan
    d6_cr_p_value = two_proprotions_test(vip_t, dnu_t_tmp, vip_c, dnu_c_tmp)[1]
    d6_cr_sig =  "Significant" if d6_cr_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": 'd6转化率', "Control": "{:.2%}".format(d6_cr_c), "Test": "{:.2%}".format(d6_cr_t), "Delta": "{:.2%}".format(d6_cr_delta), "p_value": round(d6_cr_p_value, 4), "Significance": d6_cr_sig}, ignore_index = True)
    

    test_summary['User_Group'] = user_group
    
    return test_summary

In [20]:
output = ab_summary_data(new_user_data, '全量用户')
output.style.apply(lambda x: ["background: green" if v == 'Significant' else "" for v in x], axis = 1).hide_index()

User_Group,Metrics,Control,Test,Delta,p_value,Significance
全量用户,总新增用户,"11,004","11,031",-,-,-
全量用户,d0学习率(>0m),49.69%,49.13%,-1.13%,0.408500,Non-significant
全量用户,d0有效学习率(>10m),28.98%,28.56%,-1.45%,0.486400,Non-significant
全量用户,d1有效学习率(>10m),18.59%,18.41%,-0.97%,0.728700,Non-significant
全量用户,d2有效学习率(>10m),14.10%,14.03%,-0.50%,0.879900,Non-significant
全量用户,d3有效学习率(>10m),11.42%,11.03%,-3.42%,0.358500,Non-significant
全量用户,d4有效学习率(>10m),9.12%,9.32%,2.19%,0.616500,Non-significant
全量用户,d6转化率,6.50%,6.98%,7.38%,0.153000,Non-significant


##### 分等级

In [22]:
group_list = ['A0', 'A1初', 'A1中', 'A1高', 'A2初', 'A2中', 'A2高', 'B1初', 'B1中', 'B1高', 'B2']
output = pd.DataFrame(columns = ["User_Group", "Metrics", "Control", "Test", "Delta", "p_value","Significance"])

for i in range(0,len(group_list)):
    user_group = group_list[i]
    group_data = new_user_data[new_user_data.first_milestone_name == user_group]
    output = output.append(ab_summary_data(group_data, user_group))

output.reset_index(drop=True).style.apply(lambda x: ["background: green" if v == 'Significant' else "" for v in x], axis = 1).hide_index()

User_Group,Metrics,Control,Test,Delta,p_value,Significance
A0,总新增用户,"1,396","1,308",-,-,-
A0,d0学习率(>0m),56.16%,57.87%,3.04%,0.368300,Non-significant
A0,d0有效学习率(>10m),31.73%,31.50%,-0.72%,0.895500,Non-significant
A0,d1有效学习率(>10m),16.19%,14.60%,-9.82%,0.253600,Non-significant
A0,d2有效学习率(>10m),10.53%,9.71%,-7.79%,0.479800,Non-significant
A0,d3有效学习率(>10m),6.95%,6.80%,-2.16%,0.882300,Non-significant
A0,d4有效学习率(>10m),5.59%,4.51%,-19.32%,0.202000,Non-significant
A0,d6转化率,2.15%,2.45%,13.95%,0.605500,Non-significant
A1初,总新增用户,673,739,-,-,-
A1初,d0学习率(>0m),56.61%,58.86%,3.97%,0.392300,Non-significant


### 掌握度波动

In [43]:
def ab_summary_data(data, user_group):
    test_summary = pd.DataFrame(columns = ["User_Group", "Metrics", "Control", "Test", "Delta", "p_value","Significance"])
    
    test_data = data.loc[data.user_group == '实验组']
    ctrl_data = data.loc[data.user_group == '对照组']

    dnu_t = len(test_data)
    dnu_c = len(ctrl_data)

    # 新增用户
    test_summary = test_summary.append({"Metrics": '实验用户', "Control": '{:,}'.format(dnu_c), "Test": '{:,}'.format(dnu_t), "Delta": '-', "p_value": '-', "Significance": '-'}, ignore_index = True)

    # 掌握度异常波动知识点占比
    answers_kp_video_review_t = test_data.answers_kp_video_review.sum()
    answers_kp_video_review_c = ctrl_data.answers_kp_video_review.sum()
    
    test_summary = test_summary.append({"Metrics": '答题知识点数量', "Control": '{:,}'.format(answers_kp_video_review_c), "Test": '{:,}'.format(answers_kp_video_review_t), "Delta": '-', "p_value": '-', "Significance": '-'}, ignore_index = True)

    
    answers_fluctuate_kp_video_review_t = test_data.answers_fluctuate_kp_video_review.sum()
    answers_fluctuate_kp_video_review_c = ctrl_data.answers_fluctuate_kp_video_review.sum()
    
    fl_kp_ratio_t = answers_fluctuate_kp_video_review_t / answers_kp_video_review_t
    fl_kp_ratio_c = answers_fluctuate_kp_video_review_c / answers_kp_video_review_c
    
    fl_kp_ratio_delta = fl_kp_ratio_t / fl_kp_ratio_c - 1 if fl_kp_ratio_c >0 else np.nan
    fl_kp_ratio_p_value = two_proprotions_test(answers_fluctuate_kp_video_review_t, answers_kp_video_review_t, answers_fluctuate_kp_video_review_c, answers_kp_video_review_c)[1]
        
    fl_kp_ratio_sig =  "Significant" if fl_kp_ratio_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": '掌握度异常波动知识点占比', "Control": "{:.2%}".format(fl_kp_ratio_c), "Test": "{:.2%}".format(fl_kp_ratio_t), "Delta": "{:.2%}".format(fl_kp_ratio_delta), "p_value": round(fl_kp_ratio_p_value, 4), "Significance": fl_kp_ratio_sig}, ignore_index = True)

    test_summary['User_Group'] = user_group
    
    return test_summary

In [44]:
kp_data = all_data[all_data.answers_kp_video_review >0]

output = ab_summary_data(kp_data, '全量用户')
output.style.apply(lambda x: ["background: green" if v == 'Significant' else "" for v in x], axis = 1).hide_index()

User_Group,Metrics,Control,Test,Delta,p_value,Significance
全量用户,实验用户,"4,145","4,992",-,-,-
全量用户,答题知识点数量,"610,955","486,311",-,-,-
全量用户,掌握度异常波动知识点占比,0.93%,0.62%,-33.57%,0.000000,Significant


In [36]:
kp_data.head()

,user_id,user_group,abtest_dt,first_milestone_name,new_user,new_user_vip,d0_finish_lessons,d0_study_duration,d1_study_duration,d2_study_duration,d3_study_duration,d4_study_duration,d5_study_duration,d6_study_duration,d7_study_duration,answers_kp_video_review,answers_fluctuate_kp_video_review,fluctuate_kp_ratio
9,1002392172104669,实验组,2021-02-06,A2初,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130,0,0.0
21,4503206429468903,实验组,2021-02-05,B1中,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76,0,0.0
27,721708199982973,实验组,2021-03-01,B1初,1,0,1.0,1908.0,2436.0,2385.0,2394.0,0.0,0.0,0.0,0.0,13,0,0.0
40,1865197917582813,实验组,2021-02-23,未定级,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,0,0.0
42,404238741016589,对照组,2021-02-12,B1初,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60,0,0.0


In [37]:
len(kp_data)

9137